<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/NLP_and_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

In [7]:
def softmax(scores):
  return torch.exp(scores) / torch.sum(torch.exp(scores), dim = 1)

def scaled_dot_product_attention(queries, keys, values):
  scores = (queries @ keys.T) / torch.sqrt(keys.shape[-1])
  s = softmax(scores)
  return s @ values


# class DotProductAttention(nn.Layer):
#   def __init__(self):
#     super().__init__()

#   def call(self, queries, keys, values):


In [9]:
class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim):
    super().__init__()
    self.embed = nn.Embedding(input_dim, embed_dim)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True)
    self.fc = nn.Linear(2*hidden_dim, hidden_dim)
    self.tanh = nn.Tanh()

  def forward(self, x):
    em = self.embed(x)
    lstm_out, (hidden, _) = self.lstm(em)
    return lstm_out, self.tanh(self.fc(torch.cat((hidden[0], hidden[1]), dim=1)))

class Decoder(nn.Module):
  def __init__(self, embed_dim, hidden_dim, output_dim):
    super().__init()
    self.embed = nn.Embedding(output_dim, embed_dim)
    self.lstm = nn.LSTM(hidden_dim + embed_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, targets, hidden, encoder_out):
    em = self.embed(targets)
    attn = scaled_dot_product_attention(hidden[-1], encoder_out, encoder_out)
    lstm_out, (hidden, _) = self.lstm(torch.cat((em, attn), dim=2))
    return self.fc(lstm_out), hidden

class Seq2SeqAttn(nn.Module):
  def __init__(self, enc, dec):
    super().__init__()
    self.encoder = enc
    self.decoder = dec

  def forward(self, inputs, targets):
    tar_len = targets.shape[0]
    tar_size = targets.shape[1]
    outputs = torch.zeros(tar_len, tar_size)
    enc_out, hidden = self.encoder(inputs)
    for i in range(1, tar_len):
      dec_out, hidden = self.decoder(targets[i], hidden, enc_out)
      outputs[i] = dec_out
    return outputs


In [ ]:

class PositionalEncoding(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x, seq_len, embedding_dim):
    embedding = torch.zeros((seq_len, embedding_dim))
    positions = torch.arange(seq_len)
    embedding[positions, ::2] = torch.sin(positions*(10000**(2*torch.arange(embedding_dim)[:embedding_dim//2]/embedding_dim)))
    embedding[positions, 1::2] = torch.cos(positions*(10000**(2*torch.arange(embedding_dim)[:embedding_dim//2]/embedding_dim)))
    return embedding


class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, q, k, v):
    scores = np.matmul(q, k.T) / np.sqrt(np.shape(k)[-1])
    s = softmax(scores)
    return np.matmul(s, v)

class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads):
    self.nh = num_heads


class FFN(nn.Module):
  def __init__(self, embedding_dim = 64, hidden_dim = 128):
    super().__init__()
    self.w1 = nn.Parameter(torch.randn(embedding_dim, hidden_dim))
    self.b1 = nn.Parameter(torch.randn(hidden_dim))
    self.w2 = nn.Parameter(torch.randn(hidden_dim, embedding_dim))
    self.b2 = nn.Parameter(torch.randn(embedding_dim))

  def forward(self, x):
    return (torch.max(0, (x @ self.w1) + self.b1) @ self.w2) + self.b2


class encoder(nn.Module):
  def __init__(self, embedding_dim = 64):
    super().__init__()



class decoder(nn.Module):



class Transformer(nn.Module):